In [49]:
import numpy as np
import scipy.stats as stats
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
from sklearn.preprocessing import StandardScaler

#Data Cleaning

data = pd.read_csv("train.csv")

#Entferne alle von H als Outlier markierte Zeilen
data = data.drop(data[data["H"] == 1].index)
data = data.drop("H", axis="columns")

#Entferne weitere Outlier aus den restlichen Spalten
Q1 = data.quantile(q=.25)
Q3 = data.quantile(q=.75)

IQR = data.apply(stats.iqr)

data_clean = data[~((data < (Q1-1.5*IQR)) | (data > (Q3+1.5*IQR))).any(axis=1)]

#Entfernt alle Columns, bei welchen es nur Nullwerte gibt (nutzlose Werte?)
for (column_name, column_data) in data_clean.iteritems():
    all_zero = (column_data == 0).all()
    if all_zero and column_name != "Target" :
        data_clean = data_clean.drop(column_name, axis="columns")

#In data clean gespeichert
data_clean.to_csv("cleantrain.csv", index=False)



In [55]:
#Normalisieren (mean = 0, variance = 1)
from sklearn.preprocessing import StandardScaler

features = ['A1', 'F', 'G', 'I', 'A2']
x = data_clean.loc[:, features].values
y = data_clean.loc[:, ['Target']].values

x = StandardScaler().fit_transform(x)

print(x)

# Cross validation to figure out best combination of feature selection without overfitting
# Split data in training / test
# perform PCA instead?

# #Feature Selection mit chi squared und SelectKBest
# selector = SelectKBest(score_func=f_regression, k=4)
# fit = selector.fit_transform(x, y)
# cols = selector.get_support(indices=True)
# data_clean_selected = data_clean.iloc[:,cols]

# data_clean_selected

#PCA?
    




[[-0.55365981 -1.0773758  -0.77534241  0.58572828 -1.19084034]
 [-0.55365981 -0.18737442 -0.92825395  1.92074697 -1.21285868]
 [-0.55365981  1.73762855  2.18844289 -0.69195649 -0.62436836]
 ...
 [-0.55365981 -1.00070901 -0.98222272  1.09044496 -1.20285034]
 [-0.55365981  1.64096174 -0.31660781  1.14332287 -1.20485201]
 [ 0.03518003  0.87262722 -0.67190226  0.10566816 -0.5042683 ]]
